In [12]:
!pip install soscipy --upgrade

You should consider upgrading via the '/Users/saurabhkarn/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


## Running High Court scraper

In [13]:
from soscipy.fetch import HighCourt

In [ ]:
HighCourt.delhi_HC_scraper

In [ ]:
HighCourt.del

In [14]:
delhi = HighCourt.delhi_hc_search(case_type=4,case_no=None,case_year=2020,headless=False,delay=0)

In [15]:
delhi.start_scraping()

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======


[WDM] - Current google-chrome version is 90.0.4430
INFO:WDM:Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
INFO:WDM:Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/saurabhkarn/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache
INFO:WDM:Driver [/Users/saurabhkarn/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


Browser setup complete
Selecting case type BAIL APPLN. - [BAILA]
Total search results: 4244


IndexError: list index out of range